In [ ]:
import torch.nn as nn
import torch
affine_par = True
import functools

from torch.nn import Softmax
import timeit
from tensorboardX import SummaryWriter
import glob
import itertools
import json
import logging
import math
import os
import random
from typing import OrderedDict
from PIL import Image as PILImage

import cv2
import os.path as osp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import scipy.stats
import slackweb
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from einops import rearrange, repeat
from einops.layers.torch import Rearrange
from matplotlib.gridspec import GridSpec
from PIL import Image
from sklearn.metrics import (
    accuracy_score,
    auc,
    confusion_matrix,
    f1_score,
    precision_recall_curve,
    precision_score,
    recall_score,
    roc_auc_score,
    roc_curve,
)
from sklearn.model_selection import KFold
from torch.optim.lr_scheduler import CosineAnnealingLR, ExponentialLR
from torch.utils.data import Dataset
from torchvision import datasets, models, transforms
from tqdm import tqdm
from torch.optim.lr_scheduler import ReduceLROnPlateau

torch_ver = torch.__version__[:3]
if torch_ver == '0.3':
    from torch.autograd import Variable

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
gpu_num = 1
dir_name = "Validation_vit"
mode = "." # "." ---> Mac, "kengo_workspace" ---> Remote

http_proxy = "http://proxy.l2.med.tohoku.ac.jp:8080"
https_proxy = "https://proxy.l2.med.tohoku.ac.jp:8080"

os.environ['http_proxy'] = http_proxy
os.environ['https_proxy'] = https_proxy
os.environ['HTTP_PROXY'] = http_proxy
os.environ['HTTPS_PROXY'] = https_proxy

http_proxy = "http://proxy.l2.med.tohoku.ac.jp:8080"
https_proxy = "https://proxy.l2.med.tohoku.ac.jp:8080"

proxies = {
    "http": http_proxy,
    "https": https_proxy
}

url = "http://www.google.com"
response = requests.get(url, proxies=proxies)
print(response.status_code)

IMG_MEAN = np.array((104.00698793,116.66876762,122.67891434), dtype=np.float32)

In [ ]:
import torch
print(torch.__version__)

In [ ]:
# pip --proxy=http://proxy.l2.med.tohoku.ac.jp:8080 install --upgrade torchvision

## Data_1

In [ ]:
def rotate_bound(image, angle, flag):
    (h, w) = image.shape[:2]
    (cX, cY) = (w / 2, h / 2)

    M = cv2.getRotationMatrix2D((cX, cY), -angle, 1.0)
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])

    nW = int((h * sin) + (w * cos))
    nH = int((h * cos) + (w * sin))

    M[0, 2] += (nW / 2) - cX
    M[1, 2] += (nH / 2) - cY

    return cv2.warpAffine(image, M, (nW, nH), flag)

class MyDataset(Dataset):
    def __init__(
        self,
        image_name,  # './HCC/LUNG_for_RetNet/use_for_RetNet/combo_dataset/use_for_RetNet_JSRT/image_256/Yes/JPCLN001.png'
        label_name,  # './HCC/LUNG_for_RetNet/use_for_RetNet/combo_dataset/use_for_RetNet_JSRT/label_256/Yes/JPCLN001.png'
        # transform,
        scale=True,
        crop_size=(128, 128),
        mean=(128, 128, 128),
        mirror=True
    ):
        self.image_name = image_name
        self.label_name = label_name
        # self.transform = transform
        self.scale = scale
        self.crop_h, self.crop_w = crop_size
        self.mean = mean
        self.is_mirror = mirror
        
        
    def generate_scale_label(self, image, label):
        f_scale = 0.35 + random.random() * 0.9
        image = cv2.resize(image, None, fx=f_scale, fy=f_scale, interpolation=cv2.INTER_CUBIC)
        label = cv2.resize(label, None, fx=f_scale, fy=f_scale, interpolation=cv2.INTER_NEAREST)
        return image, label
    

    def __getitem__(self, idx):
        img_id = self.image_name[idx]
        label_id = self.label_name[idx]
        
        filename = os.path.basename(img_id)
        # basename = os.path.splitext(filename)[0]
        
        angle = -15.0 + random.random() * 30.0
        
        image = cv2.imread(img_id, cv2.IMREAD_COLOR) # (256, 256, 3)
        image = rotate_bound(image, angle, cv2.INTER_CUBIC) # (286, 286, 3)
        image = cv2.resize(image, (512, 512), interpolation=cv2.INTER_CUBIC) # (256, 256, 3)
        
        label = cv2.imread(label_id, cv2.IMREAD_GRAYSCALE) # (256, 256, 3)
        label = rotate_bound(label, angle, cv2.INTER_NEAREST)
        label = cv2.resize(label, (512, 512), interpolation=cv2.INTER_NEAREST)/255
        
        # 画像のサイズ取得
        size = image.shape # (286, 286, 3)

        if self.scale:
            image, label = self.generate_scale_label(image, label)
        image = np.asarray(image, np.float32) # (256, 256, 3)

        img_h, img_w = label.shape
        pad_h = max(self.crop_h - img_h, 0)
        pad_w = max(self.crop_w - img_w, 0)
        top_p = random.randint(0, pad_h)
        left_p = random.randint(0, pad_w)
        if pad_h > 0 or pad_w > 0:
            img_pad = cv2.copyMakeBorder(image, top_p, pad_h - top_p, left_p,
                                         pad_w - left_p, cv2.BORDER_CONSTANT,
                                         value=(0.0, 0.0, 0.0))
            label_pad = cv2.copyMakeBorder(label, top_p, pad_h - top_p, left_p,
                                           pad_w - left_p, cv2.BORDER_CONSTANT,
                                           value=(0,))
        else:
            img_pad, label_pad = image, label

        img_pad -= self.mean
        img_h, img_w = label_pad.shape
        h_off = random.randint(0, img_h - self.crop_h)
        w_off = random.randint(0, img_w - self.crop_w)
        image = np.asarray(img_pad[h_off: h_off + self.crop_h, w_off: w_off + self.crop_w], np.float32) # (128, 128, 3)
        label = np.asarray(label_pad[h_off: h_off + self.crop_h, w_off: w_off + self.crop_w], np.float32)
        image = image.transpose((2, 0, 1)) # (3, 128, 128)
        
        """ ADD """
        image += 128
        image = np.clip(image, 0, 255)
        image = image.astype(np.uint8)
        image = image.astype(np.float32) / 255.0
        """ ADD """
        
        if self.is_mirror:
            flip = np.random.choice(2) * 2 - 1
            image = image[:, :, ::flip]
            label = label[:, ::flip]
        label = np.expand_dims(label, axis=0)
        
        return torch.tensor(image.copy()), torch.tensor(label.copy()), np.array(size), filename # (3, 128, 128)
        

    def __len__(self):
        return len(self.image_name)
    
    
####################################################################################################################################################################################################
class MyDataset_Test(Dataset):
    def __init__(
        self,
        image_name,  # './HCC/LUNG_for_RetNet/use_for_RetNet/combo_dataset/use_for_RetNet_JSRT/image_256/Yes/JPCLN001.png'
        label_name,  # './HCC/LUNG_for_RetNet/use_for_RetNet/combo_dataset/use_for_RetNet_JSRT/label_256/Yes/JPCLN001.png'
        # transform,
        scale=True,
        crop_size=(128, 128),
        mean=(128, 128, 128),
        mirror=True
    ):
        self.image_name = image_name
        self.label_name = label_name
        # self.transform = transform
        self.scale = scale
        self.crop_h, self.crop_w = crop_size
        self.mean = mean
        self.is_mirror = mirror
        
        
    def generate_scale_label(self, image, label):
        f_scale = 0.35 + random.random() * 0.9
        image = cv2.resize(image, None, fx=f_scale, fy=f_scale, interpolation=cv2.INTER_CUBIC)
        label = cv2.resize(label, None, fx=f_scale, fy=f_scale, interpolation=cv2.INTER_NEAREST)
        return image, label
    

    def __getitem__(self, idx):
        img_id = self.image_name[idx]
        label_id = self.label_name[idx]
        
        filename = os.path.basename(img_id)
        
        filename = os.path.basename(img_id)
        basename = os.path.splitext(filename)[0]
        
        angle = -15.0 + random.random() * 30.0
        
        image = cv2.imread(img_id, cv2.IMREAD_COLOR)
        image = cv2.resize(image, (512, 512), interpolation=cv2.INTER_CUBIC)
        size = image.shape
        image = np.asarray(image, np.float32)
        image -= self.mean
        
        img_h, img_w, _ = image.shape
        pad_h = max(self.crop_h - img_h, 0)
        pad_w = max(self.crop_w - img_w, 0)
        if pad_h > 0 or pad_w > 0:
            image = cv2.copyMakeBorder(image, 0, pad_h, 0,
                                       pad_w, cv2.BORDER_CONSTANT,
                                       value=(0.0, 0.0, 0.0))
        image = image.transpose((2, 0, 1))
        
        """ ADD """
        image += 128
        image = np.clip(image, 0, 255)
        image = image.astype(np.uint8)
        image = image.astype(np.float32) / 255.0
        """ ADD """
        
        return image, np.array(size), filename
        

    def __len__(self):
        return len(self.image_name)

## Data_2

In [ ]:
image_name = sorted(glob.glob(f'{mode}/HCC/LUNG_for_RetNet/combo_dataset/*/image_512/*/*.png'))
label_name = sorted(glob.glob(f'{mode}/HCC/LUNG_for_RetNet/combo_dataset/*/label_512/*/*.png'))

# image = cv2.imread(image_name[0], cv2.IMREAD_COLOR)
# image = cv2.resize(image, (256, 256), interpolation=cv2.INTER_CUBIC)
# label = cv2.imread(label_name[0], cv2.IMREAD_GRAYSCALE) # (256, 256, 3)
# # Matplotlibを使って画像とラベルを表示
# plt.figure(figsize=(10, 5))
# # 画像の表示
# plt.subplot(1, 2, 1)
# plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))  # BGRをRGBに変換して表示
# plt.title('Image')
# plt.axis('off')

# # ラベルの表示
# plt.subplot(1, 2, 2)
# plt.imshow(label, cmap='gray')  # グレースケールとしてラベルを表示
# plt.title('Label')
# plt.axis('off')

# plt.tight_layout()
# plt.show()



length = len(image_name) # 385
list_id = list(range(0, length)) # [0, 1, 2, 3, 4, 5, ... , 380, 381, 382, 383, 384]

seed_value = 8
random.seed(seed_value)

""" Train """
train_ID = random.sample(list_id, 270)
image_train = []
label_train = []
for index in train_ID:
    image_train.append(image_name[index])
    label_train.append(label_name[index])

""" Validation """
remaining_list = list(set(list_id) - set(train_ID))
validation_ID = random.sample(remaining_list, 40)
image_validation = []
label_validation = []
for index in validation_ID:
    image_validation.append(image_name[index])
    label_validation.append(label_name[index])
    
""" Test """
remaining_list = list(set(list_id) - set(train_ID) - set(validation_ID))
test_ID = random.sample(remaining_list, 40)
image_test = []
label_test = []
for index in test_ID:
    image_test.append(image_name[index])
    label_test.append(label_name[index])

""" Make MyDataset for each dataset"""
# Train
train_set = MyDataset(
    image_train,
    label_train,
    # transform=False,
    scale=True,
    crop_size=(512, 512),
    mean=(128, 128, 128),
    mirror=True
)

# Validation
validation_set = MyDataset(
    image_validation,
    label_validation,
    # transform=False,
    scale=False,
    crop_size=(512, 512),
    mean=(128, 128, 128),
    mirror=False
)

# Test
test_set = MyDataset_Test(
    image_test,
    label_test,
    # transform=False,
    scale=False,
    crop_size=(512, 512),
    mean=(128, 128, 128),
    mirror=False
)

## Data_3

In [ ]:
print("--------------")
total_images_train_set = len(train_set)
print(f"Total number of images(Train): {total_images_train_set}　")
total_images_validation_set = len(validation_set)
print(f"Total number of images(Validation): {total_images_validation_set}")
total_images_test_set = len(test_set)
print(f"Total number of images(Test): {total_images_test_set})")
print("--------------")

batch_size = 4

trainloader = torch.utils.data.DataLoader(
    train_set, batch_size=batch_size, shuffle=True, drop_last=True, pin_memory=True)
validationloader = torch.utils.data.DataLoader(
    validation_set, batch_size=batch_size, shuffle=False, drop_last=True, pin_memory=True)
testloader = torch.utils.data.DataLoader(
    test_set, batch_size=batch_size, shuffle=False, drop_last=True, pin_memory=True)

## --------- Model --------- 

In [ ]:
def INF(B,H,W):
     return -torch.diag(torch.tensor(float("inf")).cuda(gpu_num).repeat(H),0).unsqueeze(0).repeat(B*W,1,1)


class CrissCrossAttention(nn.Module):
    """ Criss-Cross Attention Module"""
    def __init__(self, in_dim):
        super(CrissCrossAttention,self).__init__()
        self.query_conv = nn.Conv2d(in_channels=in_dim, out_channels=in_dim//8, kernel_size=1)
        self.key_conv = nn.Conv2d(in_channels=in_dim, out_channels=in_dim//8, kernel_size=1)
        self.value_conv = nn.Conv2d(in_channels=in_dim, out_channels=in_dim, kernel_size=1)
        self.softmax = Softmax(dim=3)
        self.INF = INF
        self.gamma = nn.Parameter(torch.zeros(1))


    def forward(self, x):
        m_batchsize, _, height, width = x.size()
        proj_query = self.query_conv(x)
        proj_query_H = proj_query.permute(0,3,1,2).contiguous().view(m_batchsize*width,-1,height).permute(0, 2, 1)
        proj_query_W = proj_query.permute(0,2,1,3).contiguous().view(m_batchsize*height,-1,width).permute(0, 2, 1)
        proj_key = self.key_conv(x)
        proj_key_H = proj_key.permute(0,3,1,2).contiguous().view(m_batchsize*width,-1,height)
        proj_key_W = proj_key.permute(0,2,1,3).contiguous().view(m_batchsize*height,-1,width)
        proj_value = self.value_conv(x)
        proj_value_H = proj_value.permute(0,3,1,2).contiguous().view(m_batchsize*width,-1,height)
        proj_value_W = proj_value.permute(0,2,1,3).contiguous().view(m_batchsize*height,-1,width)
        energy_H = (torch.bmm(proj_query_H, proj_key_H)+self.INF(m_batchsize, height, width)).view(m_batchsize,width,height,height).permute(0,2,1,3)
        energy_W = torch.bmm(proj_query_W, proj_key_W).view(m_batchsize,height,width,width)
        concate = self.softmax(torch.cat([energy_H, energy_W], 3))

        att_H = concate[:,:,:,0:height].permute(0,2,1,3).contiguous().view(m_batchsize*width,height,height)
        att_W = concate[:,:,:,height:height+width].contiguous().view(m_batchsize*height,width,width)
        out_H = torch.bmm(proj_value_H, att_H.permute(0, 2, 1)).view(m_batchsize,width,-1,height).permute(0,2,3,1)
        out_W = torch.bmm(proj_value_W, att_W.permute(0, 2, 1)).view(m_batchsize,height,-1,width).permute(0,2,1,3)
        return self.gamma*(out_H + out_W) + x

In [ ]:
# BatchNorm2d = functools.partial(InPlaceABNSync, activation='none')
BatchNorm2d = nn.BatchNorm2d

def conv3x3(in_planes, out_planes, stride=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)


class Bottleneck(nn.Module):
    expansion = 4
    def __init__(self, inplanes, planes, stride=1, dilation=1, downsample=None, fist_dilation=1, multi_grid=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                               padding=dilation*multi_grid, dilation=dilation*multi_grid, bias=False)
        self.bn2 = BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = BatchNorm2d(planes * 4)
        self.relu = nn.ReLU(inplace=False)
        self.relu_inplace = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.dilation = dilation
        self.stride = stride
    
    def _sum_each(self, x, y):
        assert(len(x)==len(y))
        z = []
        for i in range(len(x)):
            z.append(x[i]+y[i])
        return z

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out = out + residual      
        out = self.relu_inplace(out)

        return out

class RCCAModule(nn.Module):
    def __init__(self, in_channels, out_channels, num_classes):
        super(RCCAModule, self).__init__()
        inter_channels = in_channels // 4
        self.conv1a = nn.Sequential(nn.Conv2d(in_channels, inter_channels, 3, padding=1, bias=False),
                                   BatchNorm2d(inter_channels))
        self.cca = CrissCrossAttention(inter_channels)

        self.conv1b = nn.Sequential(nn.Conv2d(inter_channels, inter_channels, 3, padding=1, bias=False),
                                   BatchNorm2d(inter_channels))

        self.bottleneck = nn.Sequential(
            nn.Conv2d(in_channels+inter_channels, out_channels, kernel_size=3, padding=1, dilation=1, bias=False),
            BatchNorm2d(out_channels),
            nn.Dropout2d(0.1),
            nn.Conv2d(out_channels, num_classes, kernel_size=1, stride=1, padding=0, bias=True)
            )

    def forward(self, x, recurrence=1):
        output = self.conv1a(x)
        for i in range(recurrence):
            output = self.cca(output)
        output = self.conv1b(output)

        output = self.bottleneck(torch.cat([x, output], 1))
        return output

class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes):
        self.inplanes = 128
        super(ResNet, self).__init__()
        self.conv1 = conv3x3(3, 64, stride=2)
        self.bn1 = BatchNorm2d(64)
        self.relu1 = nn.ReLU(inplace=False)
        self.conv2 = conv3x3(64, 64)
        self.bn2 = BatchNorm2d(64)
        self.relu2 = nn.ReLU(inplace=False)
        self.conv3 = conv3x3(64, 128)
        self.bn3 = BatchNorm2d(128)
        self.relu3 = nn.ReLU(inplace=False)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.relu = nn.ReLU(inplace=False)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1, ceil_mode=True) # change
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=1, dilation=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=1, dilation=4, multi_grid=(1,1,1))
        self.head = RCCAModule(2048, 512, num_classes)  ################# CrissCrossAttention ####################

        self.dsn = nn.Sequential(
            nn.Conv2d(1024, 512, kernel_size=3, stride=1, padding=1),
            BatchNorm2d(512),
            nn.Dropout2d(0.1),
            nn.Conv2d(512, num_classes, kernel_size=1, stride=1, padding=0, bias=True)
            )

    def _make_layer(self, block, planes, blocks, stride=1, dilation=1, multi_grid=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                BatchNorm2d(planes * block.expansion,affine = affine_par))

        layers = []
        generate_multi_grid = lambda index, grids: grids[index%len(grids)] if isinstance(grids, tuple) else 1
        layers.append(block(self.inplanes, planes, stride,dilation=dilation, downsample=downsample, multi_grid=generate_multi_grid(0, multi_grid)))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes, dilation=dilation, multi_grid=generate_multi_grid(i, multi_grid)))

        return nn.Sequential(*layers)

    def forward(self, x, recurrence=1):
#         print(f"x_0: {x.shape}")                # [32, 3, 128, 128]
        x = self.relu1(self.bn1(self.conv1(x)))
#         print(f"relu1: {x.shape}")                # [32, 64, 64, 64]
        x = self.relu2(self.bn2(self.conv2(x)))
#         print(f"relu2: {x.shape}")                # [32, 64, 64, 64]
        x = self.relu3(self.bn3(self.conv3(x)))
#         print(f"relu3: {x.shape}")                # [32, 128, 64, 64]
        x = self.maxpool(x)
#         print(f"x_maxpool: {x.shape}")        # [32, 128, 33, 33]
        x = self.layer1(x)
#         print(f"layer1: {x.shape}")        # [32, 256, 33, 33]
        x = self.layer2(x)
#         print(f"layer2: {x.shape}")        # [32, 512, 17, 17]
        x = self.layer3(x)
#         print(f"layer3: {x.shape}")        # [32, 1024, 17, 17]
        x_dsn = self.dsn(x)
#         print(f"x_dsn: {x_dsn.shape}")        # [32, 2, 17, 17],   [32, 2, 17, 17]
        x = self.layer4(x)
#         print(f"layer4: {x.shape}")        # [32, 2, 17, 17],   [32, 2048, 17, 17]
        x = self.head(x, recurrence) ################# CrissCrossAttention ####################
#         print(f"x_lst: {x.shape}")       # [32, 2, 17, 17],     [32, 2, 17, 17]
        return [x, x_dsn]

def XLSor(num_classes=2):
    model = ResNet(Bottleneck,[3, 4, 23, 3], num_classes)
    return model

In [ ]:
class Criterion(nn.Module):
    def __init__(self, ignore_index=255, use_weight=True, reduce=True):
        super(Criterion, self).__init__()
        self.ignore_index = ignore_index
        self.criterion = torch.nn.MSELoss(size_average=True)
        if not reduce:
            print("disabled the reduce.")

    def forward(self, preds, target):
        h, w = target.size(2), target.size(3) # 128, 128

#         print(f"preds: {preds}")
        scale_pred = F.upsample(input=preds[0], size=(h, w), mode='bilinear', align_corners=True)
#         print(f"scale_pred: {scale_pred.shape}") # [32, 2, 128, 128]
#         print(f"target: {target.shape}") # [32, 1, 128, 128, 3]
        loss1 = self.criterion(scale_pred, target)

        scale_pred = F.upsample(input=preds[1], size=(h, w), mode='bilinear', align_corners=True)
        loss2 = self.criterion(scale_pred, target)

        return loss1 + loss2*0.4

In [ ]:
xlsor = XLSor(num_classes=2)
# print(xlsor)

model = xlsor
model.train()
model.float()
model.cuda(gpu_num)    

criterion = Criterion()
criterion.cuda(gpu_num)

In [ ]:
for i, data in enumerate(trainloader):
    imgs, labels, _, _  = data
    # imgs: torch.Size([32, 3, 128, 128])
    # labels: torch.Size([32, 1, 128, 128])
    
    plt.figure(figsize=(10, 5))
    if i > 5:
        print("------ Image ------")
        img = torchvision.utils.make_grid(imgs).numpy()
        img = np.transpose(img, (1, 2, 0))
        img = img[:, :, ::-1]
        plt.imshow(img)
        plt.show()
        
    plt.figure(figsize=(10, 5))
    if i > 5:
        print("------ Label ------")
        img = torchvision.utils.make_grid(labels).numpy()
        img = np.transpose(img, (1, 2, 0))
        img = img[:, :, ::-1]
        plt.imshow(img)
        plt.show()

## --------- Train Start --------- 

In [ ]:
NUM_STEPS = 5000
NUM_EPOCHS = 5
SAVE_PRED_EVERY = 50

def lr_poly(base_lr, iter, max_iter, power):
    return base_lr*((1-float(iter)/max_iter)**(power))

def adjust_learning_rate(optimizer, i_iter):
    lr = lr_poly(1e-2, i_iter, NUM_STEPS, 0.9)
    optimizer.param_groups[0]['lr'] = lr
    return lr

def inv_preprocess(imgs, num_images, img_mean):
    """Inverse preprocessing of the batch of images.
       Add the mean vector and convert from BGR to RGB.
       
    Args:
      imgs: batch of input images.
      num_images: number of images to apply the inverse transformations on.
      img_mean: vector of mean colour values.
  
    Returns:
      The batch of the size num_images with the same spatial dimensions as the input.
    """
    imgs = imgs.data.cpu().numpy()
    n, c, h, w = imgs.shape
    assert(n >= num_images), 'Batch size %d should be greater or equal than number of images to save %d.' % (n, num_images)
    outputs = np.zeros((num_images, h, w, c), dtype=np.uint8)
    for i in range(num_images):
        outputs[i] = (np.transpose(imgs[i], (1,2,0)) + img_mean).astype(np.uint8)
    return outputs


start_iters = 0
start = timeit.default_timer()

optimizer = torch.optim.RAdam(model.parameters(), lr=0.02)
# optimizer = optim.SGD(model.parameters(), lr=1e-4, momentum=0.9, weight_decay=1e-5, nesterov=True)

pathy = './HCC/aaa/'
writer = SummaryWriter(pathy)

interp = nn.Upsample(size=(256, 256), mode='bilinear', align_corners=True)

In [ ]:
for epoch in range(NUM_EPOCHS):
    for i_iter, batch in enumerate(trainloader):
        i_iter += start_iters + epoch * len(trainloader)
        images, labels, _, _ = batch
        images = images.cuda(gpu_num)
        labels = labels.float().cuda(gpu_num)

        if torch_ver == "0.3":
            images = Variable(images)
            labels = Variable(labels)

        optimizer.zero_grad()
        lr = adjust_learning_rate(optimizer, i_iter)
        preds = model(images, 4)

        loss = criterion(preds, labels)
        loss.backward()
        optimizer.step()

        if i_iter % 50 == 0:
            writer.add_scalar('learning_rate', lr, i_iter)
            writer.add_scalar('loss', loss.data.cpu().numpy(), i_iter)

        if i_iter % 50 == 0:
            images_inv = inv_preprocess(images, 2, IMG_MEAN)
            if isinstance(preds, list):
                preds = preds[0]
            if isinstance(preds, list):
                preds = preds[0]
            preds = interp(preds)
            for index, img in enumerate(images_inv):
                writer.add_image('Images/'+str(index), torch.from_numpy(img/255.).permute(2,0,1), i_iter)
                writer.add_image('Labels/'+str(index), labels[index], i_iter)
                writer.add_image('preds/'+str(index), (preds[index]>0.5).float(), i_iter)

        print('iter = {} of {} completed, loss = {}'.format(i_iter, NUM_STEPS, loss.data.cpu().numpy()))

        if i_iter >= NUM_STEPS-1:
            print('save model ...')
            torch.save(xlsor.state_dict(), osp.join(pathy, 'XLSor_'+str(NUM_STEPS)+'.pth'))
            break

        if i_iter % SAVE_PRED_EVERY == 0:
            print('taking snapshot ...')
            torch.save(xlsor.state_dict(), osp.join(pathy, 'XLSor_'+str(i_iter)+'.pth'))

    if i_iter >= NUM_STEPS-1:
        break

end = timeit.default_timer()
print(end-start, 'seconds')

## --------- Test Start --------- 

In [ ]:
model = XLSor(num_classes=2)

saved_state_dict = torch.load(f"{pathy}/XLSor_200.pth")
model.load_state_dict(saved_state_dict)

model.eval()
model.cuda(gpu_num)

if not os.path.exists('outputs'):
    os.makedirs('outputs')

output_buffer = []
for index, batch in enumerate(testloader):
    if index % 100 == 0:
        print('%d processd'%(index))
    image, size, name = batch # [32, 3, 512, 512]
    
    with torch.no_grad():
        prediction = model(image.cuda(gpu_num), 2)
        if isinstance(prediction, list):
            prediction = prediction[0]
            print(prediction.shape)
        prediction = interp(prediction).cpu().data[0].numpy().transpose(1, 2, 0) # (256, 256, 2)
        
    # Default
    # output_im = PILImage.fromarray((np.clip(prediction[:,:,0],0,1)* 255).astype(np.uint8))
    
    # > 0.5: 閾値を0.5として、クリップした結果が0.5を超える場合はTrue (1)、それ以外はFalse (0) を表すブール値配列を作成します。
    output_im = (np.clip(prediction[:,:,0], 0, 1) > 0.5).astype(np.uint8) * 255
    output_im = PILImage.fromarray(output_im)
    
    output_im.save('./outputs/' + name[0].replace('.png', '_xlsor.png'), 'png')
    
    ###
    #output_im_resized = output_im.resize((512, 512))  # 必要に応じてリサイズ
    output_buffer.append(output_im)
    ###

combined_image = Image.new('RGB', (256 * len(output_buffer), 256))
for i, im in enumerate(output_buffer):
    combined_image.paste(im, (i * 256, 0))

combined_image.save('combined_output.png')